# Adding a new model to a Model Version Set

In this notebook we continue the example by:
- creating a new model
- uploading that model
- updating the model version set

This shows how, in a production scenario, we can continuously update the model artefact

### Load Dependencies

In [56]:
import ads
import tempfile
from ads.model import SklearnModel
from ads.model import ModelVersionSet
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from ads.common.model_metadata import UseCaseType

  from ads.common.model_metadata import UseCaseType



### Load Current Champion Model

In [ ]:
# load the model version set
mvs = ModelVersionSet.from_id(id='ocid1.datasciencemodelversionset...')

In [23]:
# view models in version set
models = mvs.models()

In [ ]:
models

In [25]:
# get model ocid of entry in model version set
model_ocid = models[0].id

In [26]:
# download original model from model catalog
downloaded_model = SklearnModel.from_model_catalog(model_ocid,artifact_dir='downloaded-model/',ignore_conda_error=True)

loop1:   0%|          | 0/4 [00:00<?, ?it/s]

### Calculate Goodness of Fit

In [27]:
import joblib

original_model = joblib.load('downloaded-model/model.joblib')

In [29]:
# load original train/test data
X_train = pd.read_csv('data/x_train.csv')
X_test = pd.read_csv('data/x_test.csv')
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')

In [30]:
# score model on test data
preds = original_model.predict(X_test)

  warnings.warn(



In [49]:
act_pred = pd.DataFrame({'actual':np.array(y_test['0']),'pred':preds})

In [48]:
print(classification_report(np.array(y_test['0']),preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       1.00      1.00      1.00        14
           2       1.00      1.00      1.00        12

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38



Note: Change dataset later to something less deterministic

### Create New and Improved Model

In [51]:
rfc = RandomForestClassifier()

In [52]:
rfc = rfc.fit(X_train,y_train)

  return fit_method(estimator, *args, **kwargs)



In [60]:
# check model performance

preds = rfc.predict(X_test)
print(classification_report(preds,y_test['0']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       1.00      1.00      1.00        14
           2       1.00      1.00      1.00        12

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38



### Save Model to Model Catalog with Version Set

In [54]:
# Create an SklearnModel object
sklearn_model = SklearnModel(estimator=rfc, artifact_dir='updated-model/')

In [57]:
sklearn_model.prepare(inference_conda_env="generalml_p311_cpu_x86_64_v1",
training_conda_env="generalml_p311_cpu_x86_64_v1",
X_sample=X_train,
y_sample=y_train,
use_case_type=UseCaseType.MULTINOMIAL_CLASSIFICATION
)

  warnings.warn("slug will be deprecated. Provide conda pack path instead.")



algorithm: RandomForestClassifier
artifact_dir:
  /Users/hsnart/Documents/Custom Demos and Blogs/Model Version Contol/updated-model:
  - - .model-ignore
    - input_schema.json
    - runtime.yaml
    - output_schema.json
    - model.joblib
    - score.py
framework: scikit-learn
model_deployment_id: null
model_id: null

In [58]:
sklearn_model.verify(X_test[:10])["prediction"]

Start loading model.joblib from model directory /Users/hsnart/Documents/Custom Demos and Blogs/Model Version Contol/updated-model ...
Model is successfully loaded.
  return pd.read_json(json_data, dtype=fetch_data_type_from_schema(input_schema_path))

  warnings.warn(



[1, 2, 2, 1, 0, 1, 0, 2, 1, 1]

In [59]:
sklearn_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [ ]:
# Save the model and add it to the model version set
project_ocid = 'ocid1.datascienceproject...'
compartment_ocid = 'ocid1.compartment...'

model_id = sklearn_model.save(compartment_id=compartment_ocid,
    project_id=project_ocid,
    display_name="Updated model",
    model_version_set=mvs,
    version_label="Version 2")

Start loading model.joblib from model directory /Users/hsnart/Documents/Custom Demos and Blogs/Model Version Contol/updated-model ...
Model is successfully loaded.
['.model-ignore', 'input_schema.json', 'runtime.yaml', 'output_schema.json', 'model.joblib', 'score.py']


loop1:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
mvs.description = "Updated champion model to Random Forest"
mvs.update()